# Multivariante del método K-Nearest Neighbors

Hasta ahora exploramos como usar un modelo de ML, el KNN simple que uso solo una caracteristica para las predicciones. Si bien esto ayudo a familiarizarse con los conceptos basicos del ML, esta claro que usar una sola caracteristica no refleja la realidad.<br>

Hay 2 formas en las que podemos retocar este modelo para tratar de mejorar la precision (mejorar el RMSE):<br>
- Incrementar el numero de atributos (**caracteristicas**) que el modelo usa para calcular similitudes cuando ranque los vecinos mas cercanos.
- Incrementar **k**, el numero de vecinos cercanos que el modelo usa.

### Eliminacion de caracteristicas no utiles y _**Normalizacion**_ de los datos.

Al seleccionar mas atributos, tenemos que tener cuidado con las columnas que no funcionan bien con la ecuacion de distancia:
- Valores no numericos (ejemplo, ciudad o estado)
    - La ecuacion de distancia euclidiana espera valores numericos.
- Valores perdidos
    - La ecuacion de la distancia euclideana espera un valor por cada observacion y atributo.
- Valores no ordinales (por ejemplo, latitud o longitud)
    - La clasifiacion por distancia euclideana no tiene sentido si todos los valores no son ordinales. 

In [5]:
import pandas as pd
import numpy as np

# Leyendo datos y acomodandolos aleatoreamente
dc_listings = pd.read_csv('dc_airbnb.csv')
dc_listings = dc_listings.loc[ np.random.permutation(len(dc_listings)) ]

# Convirtiendo la columna price a float (limpiando datos)
stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')

dc_listings.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state
2654,100%,90%,2,4,Entire home/apt,1.0,1.0,2.0,105.0,$45.00,NaN,3,1125,21,38.912156,-77.014847,Washington,20001,DC
710,99%,96%,32,3,Entire home/apt,1.0,1.0,2.0,89.0,$45.00,$100.00,4,120,75,38.919880,-77.097958,Washington,20007,DC
3418,56%,67%,1,2,Entire home/apt,1.0,1.0,1.0,125.0,NaN,NaN,3,1125,11,38.935218,-77.071037,Washington,20016,DC
3127,100%,75%,3,6,Entire home/apt,3.0,2.0,3.0,499.0,$250.00,$500.00,3,14,2,38.926051,-77.022876,Washington,20001,DC
212,100%,100%,6,6,Entire home/apt,2.0,2.0,3.0,295.0,$100.00,NaN,1,1125,43,38.905211,-77.018342,Washington,20001,DC


In [6]:
# Mostrando informacion general de los datos. 
# Como: columnas, cuantos valores no nulos contiene cada columna y el tipo de dato
print(dc_listings.info())

<class 'pandas.core.frame.DataFrame'>
Index: 3723 entries, 2654 to 2479
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   host_response_rate    3289 non-null   object 
 1   host_acceptance_rate  3109 non-null   object 
 2   host_listings_count   3723 non-null   int64  
 3   accommodates          3723 non-null   int64  
 4   room_type             3723 non-null   object 
 5   bedrooms              3702 non-null   float64
 6   bathrooms             3696 non-null   float64
 7   beds                  3712 non-null   float64
 8   price                 3723 non-null   float64
 9   cleaning_fee          2335 non-null   object 
 10  security_deposit      1426 non-null   object 
 11  minimum_nights        3723 non-null   int64  
 12  maximum_nights        3723 non-null   int64  
 13  number_of_reviews     3723 non-null   int64  
 14  latitude              3723 non-null   float64
 15  longitude             3

Como podemos observar: 
- Hay columnas que contienen valores _**NO númericos**_:
    - _room_type_
    - _city_
    - _state_
- Y columnas con valores númericos pero _**NO ordinales**_:
    - _latitude_
    - _longitude_
    - _zipcode_


A su vez tambien evitaremos cualquier columna que no describa directamente el espacio vital:
- _host_response_rate_
- _host_acceptance_rate_
- _host_listings_count_


In [7]:
# Eliminando columnas no utiles.
drop_columns = [ 'room_type', 'city', 'state', 'latitude', 'longitude', 'zipcode', 'host_response_rate', 'host_acceptance_rate', 'host_listings_count' ]
dc_listings = dc_listings.drop(drop_columns, axis=1)
dc_listings.isnull().sum()

accommodates            0
bedrooms               21
bathrooms              27
beds                   11
price                   0
cleaning_fee         1388
security_deposit     2297
minimum_nights          0
maximum_nights          0
number_of_reviews       0
dtype: int64

De las columnas restantes, hay algunas que tienen pocos valores nulos:
- _bedrooms_
- _bathrooms_
- _beds_

De las cuales podemos eliminar esas filas sin perder mucha informacion.

Pero tambien hay 2 columnas con un gran numero de valores perdidos:
- _cleaning_fee_
- _security_deposit_

En este caso no podemos eliminar simplemente las filas, ya que perderiamos mucha informacion. Por ello, vamos a eliminar <br>
por completo esas dos columas.

In [8]:
# Eliminando columnas con un gran numero de valores perdidos
dc_listings = dc_listings.drop([ 'cleaning_fee', 'security_deposit' ], axis=1)

# Eliminando filas con valores nulos
dc_listings = dc_listings.dropna(axis=0)

dc_listings.isnull().sum()

accommodates         0
bedrooms             0
bathrooms            0
beds                 0
price                0
minimum_nights       0
maximum_nights       0
number_of_reviews    0
dtype: int64

#### Normalizacion de Columnas

In [9]:
dc_listings.head()

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
2654,4,1.0,1.0,2.0,105.0,3,1125,21
710,3,1.0,1.0,2.0,89.0,4,120,75
3418,2,1.0,1.0,1.0,125.0,3,1125,11
3127,6,3.0,2.0,3.0,499.0,3,14,2
212,6,2.0,2.0,3.0,295.0,1,1125,43


Como se puede observar, hay columnas con valores con mucha varianza, valores desmesurados. Y debido a como se calcula la _distancia euclideana_ <br>
podria ocasionar errores de distancia. 
Para evitar que una sola columna tenga demasiado impacto en la distancia, podemos _**normalizar**_ todas las columnas para que tengan una media de 0 <br>
y una desviacion estandar de 1.

_La **normalización** de los valores de cada columna preserva la distribucion de los valores de cada columana a su vez que alinea las escalas._

Esta es la formúla matemática que describe la transformacion que debe aplicarse a todos los valores de la columna: $$ Z_{i} = \frac{X_{i} - \mu}{\sigma} $$

Donde:
- $X_{i}$  es in valor de una columna especifica.
- $\mu$  es la media de todos los valores de la columna.
- $\sigma$  es a desviación estandar de todos los valores de la columna.

In [10]:
# Probando la normalizacion para entender la teoria.
first_transform = dc_listings['maximum_nights'] - dc_listings['maximum_nights'].mean()
normalized_column = first_transform / first_transform.std()
print(normalized_column.head(3))

# O de otra manera
normalized_column = first_transform / dc_listings['maximum_nights'].std()
print(normalized_column.head(3))

2654   -0.016573
710    -0.016601
3418   -0.016573
Name: maximum_nights, dtype: float64
2654   -0.016573
710    -0.016601
3418   -0.016573
Name: maximum_nights, dtype: float64


##### Aplicando esta trandormacion a todas las columnas de un DataFrame
Para ello se puede usar los metodos **mean()** y **std()** del DataFrame. Al utilizar estos métodos, se utilizan las medias y las desviaciones estándar de las columnas apropiadas para cada valor del DataFrame.

In [11]:
normalized_listings = ( dc_listings - dc_listings.mean() ) / (dc_listings.std())
normalized_listings['price'] =  dc_listings['price']
normalized_listings.head(3)

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
2654,0.401366,-0.249467,-0.439151,0.297345,105.0,0.211298,-0.016573,0.201570
710,-0.097589,-0.249467,-0.439151,0.297345,89.0,0.487635,-0.016601,2.048572
3418,-0.596544,-0.249467,-0.439151,-0.546858,125.0,0.211298,-0.016573,-0.140467


### Calcúlo de la Distancia Euclideana para el _**Caso Multivariante**_

Anteriormente utilizamos dos modelos univariantes de KNN, utilizando el atributo _**accomodates**_ para el primero y _**bathrooms**_ para el segundo. <br>
Ahora entrenaremos un modelo con ambos atributos. Recordando que la ecuacion de la _distancia euclideana_ esta definida como: $$ d = \sqrt{(q_{1}-p_{1})^{2} + (q_{2}-p_{2})^{2} + ... + (q_{n}-p_{n})^{2}} $$
Y bien para hacer el calculo en Python, ahora nos apoyaremos del modulo _**scipy**_, utilizando la funcion _**distance.euclidean()**_:

In [12]:
from scipy.spatial import distance

first_listing = [ -0.596544, -0.439151 ]
second_listing = [ -0.596544, -0.412923 ]

dist = distance.euclidean( first_listing, second_listing )

La funcion euclidean(), espera:
- Dos vectores, los cuales se deben representar con un objeto tipo _lista_ (lista de Python, array de NumPy o serie de Pandas).
- Ambos deben ser unidimensionales y tener el mismo numero de elementos.

In [13]:
from scipy.spatial import distance
first_listing = normalized_listings.iloc[0][ ['accommodates', 'bathrooms'] ]
fifth_listing = normalized_listings.iloc[4][ ['accommodates', 'bathrooms'] ]

first_fifth_distance = distance.euclidean( first_listing, fifth_listing )
print(f'La distancia entre el primer elemento y el quinto de nuestro conjunto de datos es: {first_fifth_distance}')

La distancia entre el primer elemento y el quinto de nuestro conjunto de datos es: 1.9748284912308813


### Introduccion a Scikit-learn

Hasta ahora hemos construido el modelo de KNN desde cero. Pero para ser mas productivo e iterar mas rapido, podemos hacer uso <br>
de la biblioteca _sickit-learn_, la cual contiene funciones para todos los principales algoritmos de ML y un flujo de trabajo simple <br>
y unificado.<br>

El flujo de trabajo de scikit-learn consta de 4 pasos pricipales:
- Instalar el modelo de aprendizaje automatico especifico que se desea.
- Ajustar el modelo a los datos de entrenamiento.
- Utilizar el modelo para hacer predicciones.
- Evaluar la precision de las predicciones.

_**IMPORTANTE**_: <br>
Cualquier modelo que nos ayude apredecir valores numericos, como el precio de venta en nuestro caso, se conoce como **modelo de regresion**. <br>
La otra clase principal de modelos de ML supervisado se denomina de **clasificacion**, en la que intentamos predecir una etiqueta a partir <br>
de un conjunto fijo de etiquetas (por ejemplo, el tipo de sangre o el sexo).

In [14]:
from sklearn.neighbors import KNeighborsRegressor
# Algunos de los hiperparametros que le podemos configurar son: algorithm, n_neighbors, p, entre otros.
knn = KNeighborsRegressor(algorithm='brute') # Regressor hace referencia a que es un modelo de la clase regresion.

#### Ajuste de Modelo y Realizacion de predicciones

Ahora podemos ajustar (fit_method) el modelo. Para ello se utiliza el metodo **fit(X,y)** donde: 
- X: Es un objeto tipo matriz, que contiene las columnas de caracteristicas que queremos utilizar del conjunto de entrenamiento.
- y: Objeto tipo lista, que contiene los valores objetivo correctos.

In [15]:
# Dividiendo los datos normalizados en datos de entrenamiento y prueba. 
train_df = normalized_listings.iloc[0:2792]
test_df = normalized_listings.iloc[2792:]

# Seleccionando las columnas de caracteristicas que se desean usar para 'ajustar' (entrenar) el modelo.
train_features = train_df[[ 'accommodates', 'bathrooms' ]]
train_target = train_df['price']

# Ajustando modelo
knn.fit(train_features, train_target)

KNeighborsRegressor(algorithm='brute')

**NOTA**: Los datos que se le pasen al metodo fit, no deben contener valores nulos.

#### Haciendo predicciones

Ahora podemos usar el metodo _**predict()**_ para hacer las predicciones, este metodo solo tiene un parametro requerido:
- Objeto tipo matriz, que contiene las columnas caracteristicas del conjunto de datos sobre el que queremos hacer predicciones.

**IMPORTANTE**: El numero de columnas de carateristicas que se utiliza durante el entrenamiento y prueba debe coincidir o scikit-learn devolvera un error.

In [18]:
predictions = knn.predict(test_df[[ 'accommodates', 'bathrooms' ]]) #Devuelve un array NumPy
print(predictions[0:10])

[101.  136.  104.6 362.6 104.6 104.6 113.  257.6 136.  205.6]


#### Aplicando todo lo aprendido de Scikit-learn

In [21]:
from sklearn.neighbors import KNeighborsRegressor

train_df = normalized_listings.iloc[0:2792]
test_df = normalized_listings.iloc[2792:]
train_columns = ['accommodates', 'bathrooms']

# Instanciando modelo de ML
knn = KNeighborsRegressor(n_neighbors=5, algorithm='brute')

# Ajustando modelo a los datos.
knn.fit(train_df[train_columns], train_df['price'])

# Haciendo predicciones
predictions = knn.predict(test_df[train_columns])
print(predictions[:20])

[101.  136.  104.6 362.6 104.6 104.6 113.  257.6 136.  205.6 104.6  97.
 589.8 104.6 104.6 136.  136.  102.  136.   84.6]
